### Install dependencies

In [1]:
!pip install boto3 sagemaker langchain langchain-community langchain-core faiss-cpu requests opensearch-py sentence-transformers langchain-text-splitters requests-aws4auth qdrant-client -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 9.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 56.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Load CSV data from S3

In [2]:
!pwd

/home/ec2-user/SageMaker/Enterprise-RAG/notebooks


In [3]:
import boto3
import pandas as pd

s3 = boto3.client('s3')
bucket_name = 'recipes-rag'

In [4]:
# file_key = 'food_recipes.csv'

# # Download the file from s3 locally
# s3.download_file(bucket_name, file_key, '../data/food_recipes.csv')

# # Load the CSV into a DataFrame
# df = pd.read_csv('../data/food_recipes.csv')

# df.head()

In [5]:
file_key = 'recipes_w_cleaning_time_combined_features.parquet'
s3.download_file(bucket_name, file_key, f'../data/{file_key}')
df = pd.read_parquet(f'../data/{file_key}')

df.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,CookTime_Minutes,PrepTime_Minutes,TotalTime_Minutes,Combined_Features_Clean
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo...",1440,45,1485,Low-Fat Berry Blue Frozen Dessert Frozen Desse...
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...,25,240,265,Biryani Chicken Breast Make share Biryani reci...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu...",5,30,35,Best Lemonade Beverages one first Good House K...
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces...",20,1440,1460,Carina's Tofu-Vegetable Kebabs Soy/Tofu dish b...
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,...",30,20,50,Cabbage Soup Vegetable Make share Cabbage Soup...


In [6]:
df.columns

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions',
       'CookTime_Minutes', 'PrepTime_Minutes', 'TotalTime_Minutes',
       'Combined_Features_Clean'],
      dtype='object')

In [7]:
str(df.iloc[0]['Combined_Features_Clean'])

'Low-Fat Berry Blue Frozen Dessert Frozen Desserts Make share Low-Fat Berry Blue Frozen Dessert recipe Food.com. Dessert Low Protein Low Cholesterol Healthy Free Of... Summer Weeknight Freezer Easy'

### Load data into chunked documents

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [9]:
embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
text_splitter_recursive = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)

In [11]:
# Using old raw dataset

# def create_documents(df):
#     # AWS Kendra requires non empty string values for each field
#     # Make explicit to the LLM that the field is not available
#     df.fillna('Not Available', inplace=True)
    
#     documents = []
#     for index, row in df.iterrows():
#         metadata = {
#             'recipe_title': str(row['recipe_title']) if row['recipe_title'] else 'No Title Available',
#             'url': str(row['url']) if row['url'] else 'https://example.com',
#             # 'record_health': str(row['record_health']) if row['record_health'] else 'Unknown',
#             'vote_count': str(row['vote_count']) if row['vote_count'] else 'No Votes Available',
#             'rating': str(row['rating']) if row['rating'] else 'No Rating Available',
#             'cuisine': str(row['cuisine']) if row['cuisine'] else 'No Cuisine Available',
#             'course': str(row['course']) if row['course'] else 'No Course Available',
#             'diet': str(row['diet']) if row['diet'] else 'No Diet Information Available',
#             'prep_time': str(row['prep_time']) if row['prep_time'] else 'No Prep Time Available',
#             'cook_time': str(row['cook_time']) if row['cook_time'] else 'No Cook Time Available',
#             'author': str(row['author']) if row['author'] else 'No Author Available',
#             'category': str(row['category']) if row['category'] else 'No Category Available'
#         }

#         # Combine all text fields for the document content
#         text = f"{row['description']} {row['ingredients']} {row['instructions']} {row['tags']}"
#         doc = Document(page_content=text, metadata=metadata)
#         documents.append(doc)
    
#     return documents

In [12]:
# Using data after EDA with more complex fields

# def create_documents(df):
#     # Ensure non-empty string values for each field
#     df.fillna('Not Available', inplace=True)
    
#     documents = []
#     for index, row in df.iterrows():
#         metadata = {
#             'recipe_id': str(row['RecipeId']) if row['RecipeId'] else 'No ID Available',
#             'name': str(row['Name']) if row['Name'] else 'No Name Available',
#             'author_id': str(row['AuthorId']) if row['AuthorId'] else 'No Author ID Available',
#             'author_name': str(row['AuthorName']) if row['AuthorName'] else 'No Author Name Available',
#             'cook_time': str(row['CookTime']) if row['CookTime'] else 'No Cook Time Available',
#             'prep_time': str(row['PrepTime']) if row['PrepTime'] else 'No Prep Time Available',
#             'total_time': str(row['TotalTime']) if row['TotalTime'] else 'No Total Time Available',
#             'date_published': str(row['DatePublished']) if row['DatePublished'] else 'No Date Available',
#             'recipe_category': str(row['RecipeCategory']) if row['RecipeCategory'] else 'No Category Available',
#             'keywords': str(row['Keywords']) if row['Keywords'] else 'No Keywords Available',
#             'aggregated_rating': str(row['AggregatedRating']) if row['AggregatedRating'] else 'No Rating Available',
#             'review_count': str(row['ReviewCount']) if row['ReviewCount'] else 'No Reviews Available',
#             'calories': str(row['Calories']) if row['Calories'] else 'No Calories Information Available',
#             'fat_content': str(row['FatContent']) if row['FatContent'] else 'No Fat Content Available',
#             'saturated_fat_content': str(row['SaturatedFatContent']) if row['SaturatedFatContent'] else 'No Saturated Fat Content Available',
#             'cholesterol_content': str(row['CholesterolContent']) if row['CholesterolContent'] else 'No Cholesterol Content Available',
#             'sodium_content': str(row['SodiumContent']) if row['SodiumContent'] else 'No Sodium Content Available',
#             'carbohydrate_content': str(row['CarbohydrateContent']) if row['CarbohydrateContent'] else 'No Carbohydrate Content Available',
#             'fiber_content': str(row['FiberContent']) if row['FiberContent'] else 'No Fiber Content Available',
#             'sugar_content': str(row['SugarContent']) if row['SugarContent'] else 'No Sugar Content Available',
#             'protein_content': str(row['ProteinContent']) if row['ProteinContent'] else 'No Protein Content Available',
#             'recipe_servings': str(row['RecipeServings']) if row['RecipeServings'] else 'No Servings Information Available',
#             'recipe_yield': str(row['RecipeYield']) if row['RecipeYield'] else 'No Yield Information Available'
#         }

#         # Combine relevant text fields for the document content
#         text = f"""
#         Name: {row['Name']}
#         Category: {row['RecipeCategory']}
#         Description: {row['Description']}
#         Keywords: {row['Keywords']}
#         Ingredients: {row['RecipeIngredientParts']}
#         Instructions: {row['RecipeInstructions']}
#         """

#         doc = Document(page_content=text.strip(), metadata=metadata)
#         documents.append(doc)
    
#     return documents


In [13]:
def create_documents(df):
    documents = []
    for index, row in df.iterrows():
        metadata = {
            'recipe_id': str(row['RecipeId']) if not pd.isna(row['RecipeId']) else 'No ID Available',
            'name': str(row['Name']) if not pd.isna(row['Name']) else 'No Name Available',
            'cook_time': str(row['CookTime']) if not pd.isna(row['CookTime']) else 'No Cook Time Available',
            'prep_time': str(row['PrepTime']) if not pd.isna(row['PrepTime']) else 'No Prep Time Available',
            'total_time': str(row['TotalTime']) if not pd.isna(row['TotalTime']) else 'No Total Time Available',
            'recipe_category': str(row['RecipeCategory']) if not pd.isna(row['RecipeCategory']) else 'No Category Available',
            'keywords': str(row['Keywords']) if not pd.isna(row['Keywords']).all() else 'No Keywords Available',
            'aggregated_rating': str(row['AggregatedRating']) if not pd.isna(row['AggregatedRating']) else 'No Rating Available',
            'review_count': str(row['ReviewCount']) if not pd.isna(row['ReviewCount']) else 'No Reviews Available',
            'calories': str(row['Calories']) if not pd.isna(row['Calories']) else 'No Calories Information Available',
            'fat_content': str(row['FatContent']) if not pd.isna(row['FatContent']) else 'No Fat Content Available',
            'saturated_fat_content': str(row['SaturatedFatContent']) if not pd.isna(row['SaturatedFatContent']) else 'No Saturated Fat Content Available',
            'cholesterol_content': str(row['CholesterolContent']) if not pd.isna(row['CholesterolContent']) else 'No Cholesterol Content Available',
            'sodium_content': str(row['SodiumContent']) if not pd.isna(row['SodiumContent']) else 'No Sodium Content Available',
            'carbohydrate_content': str(row['CarbohydrateContent']) if not pd.isna(row['CarbohydrateContent']) else 'No Carbohydrate Content Available',
            'sugar_content': str(row['SugarContent']) if not pd.isna(row['SugarContent']) else 'No Sugar Content Available',
            'protein_content': str(row['ProteinContent']) if not pd.isna(row['ProteinContent']) else 'No Protein Content Available',
            'recipe_servings': str(row['RecipeServings']) if not pd.isna(row['RecipeServings']) else 'No Servings Information Available',
            'recipe_yield': str(row['RecipeYield']) if not pd.isna(row['RecipeYield']) else 'No Yield Information Available'
        }

        # Use Combined_Features_Clean for the document content
        text = str(row['Combined_Features_Clean'])
        doc = Document(page_content=text, metadata=metadata)
        documents.append(doc)
        
    return documents


In [14]:
documents = create_documents(df)

In [15]:
documents[0]

Document(metadata={'recipe_id': '38.0', 'name': 'Low-Fat Berry Blue Frozen Dessert', 'cook_time': 'PT24H', 'prep_time': 'PT45M', 'total_time': 'PT24H45M', 'recipe_category': 'Frozen Desserts', 'keywords': "['Dessert' 'Low Protein' 'Low Cholesterol' 'Healthy' 'Free Of...' 'Summer'\n 'Weeknight' 'Freezer' 'Easy']", 'aggregated_rating': '4.5', 'review_count': '4.0', 'calories': '170.9', 'fat_content': '2.5', 'saturated_fat_content': '1.3', 'cholesterol_content': '8.0', 'sodium_content': '29.8', 'carbohydrate_content': '37.1', 'sugar_content': '30.2', 'protein_content': '3.2', 'recipe_servings': '4.0', 'recipe_yield': 'No Yield Information Available'}, page_content='Low-Fat Berry Blue Frozen Dessert Frozen Desserts Make share Low-Fat Berry Blue Frozen Dessert recipe Food.com. Dessert Low Protein Low Cholesterol Healthy Free Of... Summer Weeknight Freezer Easy')

In [16]:
len(documents)

522517

In [17]:
def split_documents_with_metadata(documents, text_splitter):
    split_docs = []
    for doc in documents:
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            split_docs.append(Document(page_content=chunk, metadata={**doc.metadata, "chunk_id": i}))
    return split_docs

In [18]:
split_documents = split_documents_with_metadata(documents, text_splitter_recursive)

In [19]:
split_documents[0]

Document(metadata={'recipe_id': '38.0', 'name': 'Low-Fat Berry Blue Frozen Dessert', 'cook_time': 'PT24H', 'prep_time': 'PT45M', 'total_time': 'PT24H45M', 'recipe_category': 'Frozen Desserts', 'keywords': "['Dessert' 'Low Protein' 'Low Cholesterol' 'Healthy' 'Free Of...' 'Summer'\n 'Weeknight' 'Freezer' 'Easy']", 'aggregated_rating': '4.5', 'review_count': '4.0', 'calories': '170.9', 'fat_content': '2.5', 'saturated_fat_content': '1.3', 'cholesterol_content': '8.0', 'sodium_content': '29.8', 'carbohydrate_content': '37.1', 'sugar_content': '30.2', 'protein_content': '3.2', 'recipe_servings': '4.0', 'recipe_yield': 'No Yield Information Available', 'chunk_id': 0}, page_content='Low-Fat Berry Blue Frozen Dessert Frozen Desserts Make share Low-Fat Berry Blue Frozen Dessert recipe Food.com. Dessert Low Protein Low Cholesterol Healthy Free Of... Summer Weeknight Freezer Easy')

In [20]:
# from transformers import AutoTokenizer

# def count_tokens(text, model_name="distilbert-base-uncased"):
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
    
#     tokens = tokenizer.encode(text, add_special_tokens=False)
#     num_tokens = len(tokens)
    
#     return num_tokens

In [21]:
# We have > 500,000 recipes, this takes a long time to run
from langchain_community.vectorstores import Qdrant

qdrant_store = Qdrant.from_documents(documents,
    embedding_model,
    location=":memory:",
)

In [22]:
qdrant_retriever = qdrant_store.as_retriever()

In [23]:
def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        formatted_docs.append(f"Metadata: {doc.metadata}\n")
    content = "\n\n".join(formatted_docs)
    
    return content

### Agent function definitions

In [24]:
recipe_db_query_tool = {
  "name": "query_food_recipe_vector_db",
  "description": """
      Queries the vector database containing food recipes to retrieve the most relevant documents. 
      This function allows the model to generate and execute multiple queries as necessary to gather comprehensive context, 
      such as ingredients, preparation steps, and metadata like cuisine and diet type, ensuring accurate and thorough responses to user queries.
      """,
  "input_schema": {
    "type": "object",
    "properties": {
      "queries": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "A query generated by the model to run against the vector database to fetch recipe documents."
        },
        "description": "A list of queries generated by the model to run against the vector database to fetch recipe documents."
      }
    },
    "required": ["queries"]
  }
}


### Init bedrock model, define util to stateless messaging, no fn calling

In [25]:
import json

In [26]:
bedrock_client = boto3.client('bedrock-runtime', region_name="us-east-1")

In [239]:
# We will need to tune these prompts
# query_bedrock_llm() definition NEEDS TO BE RERUN
# each time when changes are made to this prompt

baseline_sys_prompt = """
You are a helpful assistant and expert in cooking recipes.

Before answering, always make at least one call to query_food_recipe_vector_db
to retrieve the relevant context of recipes and ingredients to generate an informed
and high-quality response to the user prompt but NEVER exceed a MAXIMUM of 
3 calls to the query_food_recipe_vector_db function.

Provide a response to the user prompt about food with recommended recipes and instructions.
"""

In [240]:
# We will need to tune these prompts

baseline_sys_prompt_basic = """
You are a helpful assistant and expert in cooking recipes.
You will be provided a list of relevant context about relevant food recipes
which includes metadata like ingredients and cooking instructions.

Provide a response to the user prompt about food with recommended recipes and instructions.
"""

In [241]:
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

def query_bedrock_llm(messages):
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'system': baseline_sys_prompt,
            'messages': messages,
            'max_tokens': 3000,
            "tools": [recipe_db_query_tool],

            # This config forces the model to always call the recipe db query tool atleast once 
            # https://docs.anthropic.com/en/docs/build-with-claude/tool-use#controlling-claudes-output
            # "tool_choice": {
            #     "type": "tool",
            #     "name": recipe_db_query_tool['name']
            # },
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9
        })
    )
    response_body = json.loads(response.get('body').read())
    
    return response_body

In [258]:
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

def query_bedrock_llm_basic(messages):
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'system': baseline_sys_prompt_basic,
            'messages': messages,
            'max_tokens': 3000,
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9
        })
    )
    response_body = json.loads(response.get('body').read())
    
    print(response_body['content'][0]['text'])
    
    return response_body['content'][0]['text']

### Pipe langchain together

In [245]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap

In [246]:
baseline_user_prompt_basic = """
### Here is a context:
{context} 

### Here is a user prompt:
{query}
"""

In [247]:
baseline_user_prompt = """
### Here is a user prompt:
{query}
"""

In [248]:
def process_prompt(query_args):
    prompt_with_query = baseline_user_prompt.replace("{query}", query_args['query'])
    
    # This format doesn't matter much now, but we will use it later to 
    # persist chat history for continuous dialogue
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_with_query
                }
            ]
        }
    ]
    
    return messages

In [249]:
def process_prompt_basic(query_args):
    prompt_with_context = baseline_user_prompt_basic.replace("{context}", query_args['context'])
    prompt_with_query = prompt_with_context.replace("{query}", query_args['query'])
    
    # This format doesn't matter much now, but we will use it later to 
    # persist chat history for continuous dialogue
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_with_query
                }
            ]
        }
    ]
    
    return messages

In [250]:
qdrant_rag_chain = (
    RunnableMap(
        # {"context": qdrant_retriever | format_docs,
         {"query": RunnablePassthrough()}
    )
    | process_prompt
    | query_bedrock_llm
    # | parse_event_stream
)

In [251]:
qdrant_rag_chain_basic = (
    RunnableMap(
        {"context": qdrant_retriever | format_docs,
         "query": RunnablePassthrough()}
    )
    | process_prompt_basic
    | query_bedrock_llm_basic
)

### Model generates dynamic context queries to vector db

In [252]:
test_query_1 = "I enjoy asian fusion food and I am a vegetarian. Give me one recipe with ingredients and instructions"

In [253]:
qdrant_rag_chain.invoke(test_query_1)

{'id': 'msg_bdrk_01HbdBm8vBPJTALKFoEYwcZV',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'text',
   'text': 'Okay, let me query the recipe database to find a relevant Asian fusion vegetarian recipe for you.'},
  {'type': 'tool_use',
   'id': 'toolu_bdrk_01AbEWq8GkhHEyKgFjWbZCtM',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['asian fusion vegetarian recipes']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 472, 'output_tokens': 87}}

In [35]:
test_query_2 = """
I have a peanut allergy but I like thai food. 
I also don't enjoy spicy food much, and want a meal with low carbs. 
Give a recipe with ingredients and instructions
"""

In [36]:
qdrant_rag_chain.invoke(test_query_2)

{'id': 'msg_bdrk_01VBdfhyCQep6QA1NaHE1yf7',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'text',
   'text': 'Okay, let me query the vector database for relevant Thai recipes that are peanut-free, low in carbs, and not too spicy.'},
  {'type': 'tool_use',
   'id': 'toolu_bdrk_01TQ9n6yKVMi45h55Qqnr8DY',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['thai recipes',
     'peanut free',
     'low carb',
     'mild spice']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 496, 'output_tokens': 111}}

In [37]:
test_query_3 = """
Suggest a low-carb breakfast recipe that includes eggs and spinach, 
can be prepared in under 20 minutes, 
and is suitable for a keto diet.
"""

In [38]:
qdrant_rag_chain.invoke(test_query_3)

{'id': 'msg_bdrk_01Qp5C1L7tofuVb29VgKpPuh',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'tool_use',
   'id': 'toolu_bdrk_01Khe59KyLhYxicyxio8Kkkw',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['low-carb breakfast recipe with eggs and spinach',
     'keto breakfast recipe with eggs and spinach']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 491, 'output_tokens': 82}}

In [39]:
test_query_4 = """
Suggest a healthy dinner recipe for two people that includes fish, 
is under 500 calories per serving, 
and can be made in less than 40 minutes.
"""

In [40]:
qdrant_rag_chain.invoke(test_query_4)

{'id': 'msg_bdrk_01JUtHi9TRMJgQrgRVKSCh2E',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'text',
   'text': 'Okay, let me query the recipe database to find some relevant healthy fish dinner recipes that meet the criteria.'},
  {'type': 'tool_use',
   'id': 'toolu_bdrk_01KFY1hodJJYGT3ccn2RHAjd',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['healthy fish dinner recipe under 500 calories quick']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 490, 'output_tokens': 93}}

### Implement continuous dialogue and function calling

In [41]:
def generate_message(prompt):
    if type(prompt) != str:
        raise ValueError(f'Tried to call message generate_message with non-string input: {prompt}')
        
    return {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": prompt
            }
        ]
    }

In [42]:
def generate_tool_message(fn_results):
    
    return {
        "role": "user",
        "content": fn_results
    }

In [43]:
import json

# Adds the current prompt as a new message to the chat history
# and calls bedrock with the entire chat history
# Returns the response body, llm's message, and new chat history

'''
Example response body structure:
{
   "id":"msg_bdrk_01C5GGkafK7aL3P5i3rsMr1p",
   "type":"message",
   "role":"assistant",
   "model":"claude-3-sonnet-20240229",
   "content":[
      {
         "type":"tool_use",
         "id":"toolu_bdrk_01CQiYa8BMJfpJC68DuRdwQn",
         "name":"query_food_recipe_vector_db",
         "input":{
            "queries":[
               "healthy fish dinner recipe under 500 calories",
               "fish dinner recipe for two under 40 minutes"
            ]
         }
      }
   ],
   "stop_reason":"tool_use",
   "stop_sequence":"None",
   "usage":{
      "input_tokens":559,
      "output_tokens":55
   }
}
'''
def message_handler(existing_chat_history, prompt, is_tool_message=False):
    # Fn results is an array of tool response objects
    # message structure needs to reflect that
    if is_tool_message:
        user_message = generate_tool_message(prompt)
    else:
        user_message = generate_message(prompt)
    existing_chat_history.append(user_message)

    # Parse the response content
    response_body = query_bedrock_llm(existing_chat_history)
    llm_message = {
        'role': response_body['role'],
        'content': response_body['content']
    }

    # Add the response message to the chat history
    existing_chat_history.append(llm_message)
    
    return [response_body, llm_message, existing_chat_history]

In [44]:
# Executes a list of queries and returns a list of document results
def handle_vector_db_queries(queries, retriever=qdrant_retriever): 
    context_docs = []
    for query in queries:
        query_results = retriever.invoke(query)
        context_docs.extend(query_results)

    return context_docs

In [45]:
# Takes as an argument to LLM message content, returns a list of the fn result objects
def handle_function_calls(tool_call_message_content):
    tool_results = []
    
    for tool_call in tool_call_message_content:
        # Only process messages from the LLM that are function calls
        if tool_call['type'] != 'tool_use':
            continue
        fn_id = tool_call['id']
        fn_name = tool_call['name']
        fn_args = tool_call['input']
        fn_result = {
            "type": "tool_result",
            "tool_use_id": fn_id,
        }   

        if fn_name == 'query_food_recipe_vector_db':
            if 'queries' not in fn_args:
                print(f"ERROR: Tried to call {fn_name} with invalid args {fn_args}, skipping..")
                fn_result['content'] = ""
                fn_result['is_error'] = True
                tool_results.append(fn_result)
                continue
                
            print(f"Model called {fn_name} with args {fn_args}")
            context_docs = handle_vector_db_queries(fn_args['queries'])
            context_str = format_docs(context_docs)
            fn_result['content'] = context_str
            tool_results.append(fn_result)
            
        # TODO: handle web search invocation here
        
        else:
            print(f"ERROR: Attempted call to unknown function {fn_name}")
            fn_result['content'] = ""
            fn_result['is_error'] = True
            tool_results.append(fn_result)

    return tool_results

### Run dynamic query function calling with test queries

In [105]:
'''
Example payload structure of response_body:

{'id': 'msg_bdrk_01REesjegNiLteurBoxW7pSt',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'tool_use',
   'id': 'toolu_bdrk_01191W2FuAFTRoDqKKeJSmmn',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['thai food',
     'peanut free',
     'low carb',
     'not spicy']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 573, 'output_tokens': 52}}


Example payload structure of llm_message['content']:

[{'type': 'tool_use',
   'id': 'toolu_bdrk_01191W2FuAFTRoDqKKeJSmmn',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['thai food',
     'peanut free',
     'low carb',
     'not spicy']}}]
'''

# This function is the entry point to invoke the LLM with support for function calling
# parsing output, calling requested functions, sending output is handled here
def run_chat_loop(prompt):
    print(f"[User]: {prompt}")
    response_body, llm_message, chat_history = message_handler(existing_chat_history=[], prompt=prompt)
    
    
    # The model wants to call tools, call them, provide response, repeat until content is generated
    while response_body['stop_reason'] == 'tool_use':
        fn_results = handle_function_calls(tool_call_message_content=llm_message['content'])

        # Send function results back to LLM as a new message with the existing chat history
        response_body, llm_message, chat_history = message_handler(
            existing_chat_history=chat_history, 
            prompt=fn_results,
            is_tool_message=True
        )

    # The model is done calling tools
    print(f"\n[Model]: {llm_message['content'][0]['text']}")
    return f"\n[Model]: {llm_message['content'][0]['text']}"


In [106]:
test_query_response_4 = run_chat_loop(test_query_4)

[User]: 
Suggest a healthy dinner recipe for two people that includes fish, 
is under 500 calories per serving, 
and can be made in less than 40 minutes.

Model called query_food_recipe_vector_db with args {'queries': ['healthy dinner recipe fish under 500 calories 40 minutes']}

[Model]: Based on the recipe metadata retrieved, I recommend the following healthy dinner recipe for two people that includes fish, is under 500 calories per serving, and can be made in less than 40 minutes:

"My Favorite Fish Recipe"

Ingredients:
- 2 (4 oz) fish fillets (such as tilapia, cod, or salmon)
- 1 tbsp olive oil
- 1 tsp lemon juice
- 1 tsp Italian seasoning
- Salt and pepper to taste
- 1 cup cooked quinoa
- 1 cup steamed broccoli

Instructions:
1. Preheat oven to 400°F.
2. Place the fish fillets in a baking dish and drizzle with olive oil, lemon juice, Italian seasoning, salt, and pepper.
3. Bake for 20-25 minutes, until the fish is cooked through and flakes easily with a fork.
4. Serve the baked f

In [107]:
test_query_response_3 = run_chat_loop(test_query_3)

[User]: 
Suggest a low-carb breakfast recipe that includes eggs and spinach, 
can be prepared in under 20 minutes, 
and is suitable for a keto diet.

Model called query_food_recipe_vector_db with args {'queries': ['low carb breakfast recipe with eggs and spinach', 'keto breakfast recipe with eggs and spinach', 'quick breakfast recipe with eggs and spinach']}

[Model]: Based on the recipe information retrieved, I would recommend the "Easy Eggs and Spinach Bake" recipe. This dish meets all the criteria you specified:

- It is a low-carb breakfast recipe that includes eggs and spinach.
- It can be prepared in under 20 minutes (total time of 25 minutes).
- It is suitable for a keto diet, with only 1.5g of carbs per serving.

The recipe instructions are as follows:

Ingredients:
- 6 large eggs
- 1/2 cup heavy cream
- 1/4 cup grated Parmesan cheese
- 1/4 teaspoon salt
- 1/4 teaspoon black pepper
- 2 cups fresh spinach, chopped

Instructions:
1. Preheat oven to 375°F. Grease a 9-inch pie dish

In [108]:
test_query_response_2 = run_chat_loop(test_query_2)

[User]: 
I have a peanut allergy but I like thai food. 
I also don't enjoy spicy food much, and want a meal with low carbs. 
Give a recipe with ingredients and instructions

Model called query_food_recipe_vector_db with args {'queries': ['thai food recipes without peanuts', 'low carb thai recipes', 'non-spicy thai recipes']}

[Model]: Based on the search results, here is a recipe for a low-carb, non-spicy Thai dish that avoids peanuts:

Thai Shrimp and Cabbage Stir-Fry (Low Carb)

Ingredients:
- 1 lb shrimp, peeled and deveined
- 1 head of cabbage, shredded 
- 2 tbsp coconut oil
- 2 cloves garlic, minced
- 1 tbsp fish sauce
- 1 tbsp lime juice
- 1 tsp coconut aminos (or low-sodium soy sauce)
- Salt and pepper to taste

Instructions:
1. Heat the coconut oil in a large skillet or wok over medium-high heat.
2. Add the minced garlic and sauté for 1 minute until fragrant.
3. Add the shrimp and stir-fry for 2-3 minutes until they start to turn pink.
4. Add the shredded cabbage and continue s

In [109]:
test_query_response_1 = run_chat_loop(test_query_1)

[User]: I enjoy asian fusion food and I am a vegetarian. Give me one recipe with ingredients and instructions
Model called query_food_recipe_vector_db with args {'queries': ['asian fusion vegetarian recipe']}

[Model]: Based on the search results, here is a recipe for a delicious vegetarian Asian-inspired dish:

Vegetarian Thai Curry

Ingredients:
- 1 tablespoon coconut oil
- 1 onion, diced 
- 3 cloves garlic, minced
- 1 tablespoon grated ginger
- 2 tablespoons Thai red curry paste
- 1 (13.5 oz) can coconut milk
- 2 cups mixed vegetables (such as bell peppers, broccoli, carrots)
- 1 cup cooked chickpeas or tofu
- 1 tablespoon soy sauce or tamari
- 1 teaspoon brown sugar
- Juice of 1 lime
- Salt and pepper to taste
- Chopped cilantro for garnish

Instructions:
1. In a large skillet or wok, heat the coconut oil over medium heat. Add the onion and sauté for 3-4 minutes until translucent.
2. Add the garlic and ginger and cook for 1 minute until fragrant.
3. Stir in the Thai red curry paste

### Try with higher complexity / more restrictive prompts

In [110]:
test_query_5 = """
I am on a ketogenic diet and need a dinner recipe that is dairy-free, 
low in sodium, and takes less than an hour to cook.
"""

test_query_6 = """
I'm looking for a pescatarian main course that is low in saturated fat, 
uses Asian flavors, and can be prepared in under 45 minutes.
"""

test_query_7 = """
I need a diabetic-friendly, vegan breakfast recipe that is gluten-free, 
nut-free, and low in cholesterol, but also rich in omega-3 fatty acids 
and can be prepared the night before.
"""

test_query_8 = """
I am following a strict paleo diet and need a lunch recipe that is dairy-free, 
gluten-free, low in carbs, and low in sodium. Additionally, it should be rich in antioxidants, 
and can be made in under 30 minutes with minimal cooking equipment.
"""

In [111]:
test_query_response_5 = run_chat_loop(test_query_5)

[User]: 
I am on a ketogenic diet and need a dinner recipe that is dairy-free, 
low in sodium, and takes less than an hour to cook.

Model called query_food_recipe_vector_db with args {'queries': ['keto dairy-free low sodium dinner recipe']}

[Model]: Based on the search results, the "Low Carb Chicken Salad" recipe looks like a good option that meets your dietary requirements. Here are the key details:

Ingredients:
- Cooked chicken breast, shredded or diced
- Celery, diced
- Red onion, diced 
- Mayonnaise (use a keto-friendly, dairy-free version)
- Lemon juice
- Salt and pepper to taste

Instructions:
1. In a large bowl, combine the shredded or diced chicken, celery, and red onion.
2. Add a few tablespoons of mayonnaise and a squeeze of lemon juice. Start with a small amount of mayo and add more to reach your desired consistency.
3. Season with salt and pepper to taste.
4. Stir everything together until well combined.
5. Serve the chicken salad on its own, over a bed of greens, or wit

In [112]:
test_query_response_6 = run_chat_loop(test_query_6)

[User]: 
I'm looking for a pescatarian main course that is low in saturated fat, 
uses Asian flavors, and can be prepared in under 45 minutes.

Model called query_food_recipe_vector_db with args {'queries': ['pescatarian main course', 'low saturated fat', 'asian flavors', 'under 45 minutes']}

[Model]: Based on the search results, here is a recommended pescatarian main course recipe that meets the criteria:

Roasted Cauliflower With Asian Flavors

Ingredients:
- 1 head cauliflower, cut into florets
- 2 tbsp olive oil
- 2 tbsp low-sodium soy sauce
- 1 tbsp rice vinegar
- 1 tsp sesame oil
- 1 tsp grated ginger
- 1 clove garlic, minced
- 1/4 tsp red pepper flakes (optional for spice)
- Salt and pepper to taste
- Chopped green onions and sesame seeds for garnish

Instructions:
1. Preheat oven to 400°F. Toss the cauliflower florets with the olive oil, soy sauce, rice vinegar, sesame oil, ginger, garlic, and red pepper flakes (if using) on a large baking sheet. Season with salt and pepper.
2

In [113]:
test_query_response_7 = run_chat_loop(test_query_7)

[User]: 
I need a diabetic-friendly, vegan breakfast recipe that is gluten-free, 
nut-free, and low in cholesterol, but also rich in omega-3 fatty acids 
and can be prepared the night before.

Model called query_food_recipe_vector_db with args {'queries': ['diabetic-friendly vegan breakfast recipe', 'gluten-free nut-free low cholesterol breakfast', 'omega-3 rich breakfast recipe']}

[Model]: Based on the recipe metadata retrieved, here is a recommended diabetic-friendly, vegan, gluten-free, nut-free, low-cholesterol breakfast recipe that is rich in omega-3s and can be prepared the night before:

Baked Oatmeal with Omega-3s

Ingredients:
- 2 cups rolled oats (gluten-free)
- 1/4 cup ground flaxseed (for omega-3s)
- 1 tsp baking powder
- 1/2 tsp cinnamon
- 1/4 tsp salt
- 1 1/2 cups unsweetened almond milk
- 1/4 cup maple syrup
- 1 tsp vanilla extract
- 1 ripe banana, mashed
- 1/2 cup frozen blueberries

Instructions:
1. Preheat oven to 375°F. Grease an 8x8 inch baking dish.
2. In a large 

In [114]:
test_query_response_8 = run_chat_loop(test_query_8)

[User]: 
I am following a strict paleo diet and need a lunch recipe that is dairy-free, 
gluten-free, low in carbs, and low in sodium. Additionally, it should be rich in antioxidants, 
and can be made in under 30 minutes with minimal cooking equipment.

Model called query_food_recipe_vector_db with args {'queries': ['paleo lunch recipe', 'low carb low sodium recipe', 'quick easy paleo recipe']}

[Model]: Based on the recipe metadata retrieved, here is a recommended paleo-friendly, low-carb, low-sodium lunch recipe that is quick and easy to make:

Paleo Sandwich Bread

Ingredients:
- 3 eggs
- 1/2 cup almond flour
- 1/4 cup coconut flour 
- 1/4 cup olive oil
- 1 tsp baking soda
- 1/4 tsp salt

Instructions:
1. Preheat oven to 350°F. Grease a 9x5 inch loaf pan.
2. In a medium bowl, whisk the eggs. Then mix in the almond flour, coconut flour, olive oil, baking soda, and salt until well combined.
3. Pour the batter into the prepared loaf pan and bake for 40 minutes, until a toothpick insert

### Basic RAG Response

In [261]:
test_query_response_1_basic_rag = qdrant_rag_chain_basic.invoke(test_query_1)
print(test_query_response_1_basic_rag)

Based on your preference for Asian fusion vegetarian dishes, I would recommend the Vegetarian Thai Curry recipe. Here are the ingredients and instructions:

Ingredients:
- 1 tablespoon vegetable oil
- 1 onion, diced
- 3 cloves garlic, minced
- 1 tablespoon grated fresh ginger
- 2 tablespoons red curry paste
- 1 (13.5 ounce) can coconut milk
- 1 cup vegetable broth
- 1 red bell pepper, sliced
- 1 cup sliced mushrooms
- 1 cup sliced carrots
- 1 cup broccoli florets
- 1 cup snow peas
- 2 tablespoons soy sauce
- 1 tablespoon lime juice
- Salt and pepper to taste
- Cooked rice, for serving

Instructions:

1. Heat the vegetable oil in a large skillet or wok over medium-high heat.
2. Add the onion and sauté for 2-3 minutes until translucent.
3. Add the garlic and ginger, and cook for 1 minute until fragrant.
4. Stir in the red curry paste and cook for 2 minutes.
5. Pour in the coconut milk and vegetable broth, and bring to a simmer.
6. Add the red bell pepper, mushrooms, carrots, broccoli, an

In [262]:
test_query_response_2_basic_rag = qdrant_rag_chain_basic.invoke(test_query_2)
print(test_query_response_2_basic_rag)

Based on your preferences for Thai food without peanuts, low spice, and low carbs, I would recommend the Thai Shrimp and Cabbage Stir-Fry recipe. Here are the ingredients and instructions:

Ingredients:
- 1 tablespoon sesame oil
- 1 pound shrimp, peeled and deveined
- 1 tablespoon fish sauce
- 1 tablespoon soy sauce
- 1 teaspoon rice vinegar
- 1/2 teaspoon garlic powder
- 1/4 teaspoon ground ginger
- 1/4 teaspoon red pepper flakes (or less if you prefer less spice)
- 4 cups shredded cabbage
- 1/2 cup sliced green onions

Instructions:

1. Heat the sesame oil in a large skillet or wok over high heat.
2. Add the shrimp and stir-fry for 2-3 minutes until they start to turn pink.
3. In a small bowl, whisk together the fish sauce, soy sauce, rice vinegar, garlic powder, ginger, and red pepper flakes.
4. Pour the sauce mixture over the shrimp and add the shredded cabbage and green onions.
5. Continue to stir-fry for 2-3 minutes more until the cabbage is wilted and the shrimp are fully cooked

In [263]:
test_query_response_3_basic_rag = qdrant_rag_chain_basic.invoke(test_query_3)
print(test_query_response_3_basic_rag)

Based on the provided context, the Spinach Omelette recipe seems to be the perfect fit for your request. It is a low-carb breakfast option that includes eggs and spinach, can be prepared in under 20 minutes (total time is 20 minutes), and is suitable for a keto diet as it is categorized as "Very Low Carbs".

Here are the details for the Spinach Omelette recipe:

Ingredients:
- Eggs
- Spinach
- Cheese

Instructions:
1. Prepare the spinach and any other desired fillings.
2. Beat the eggs in a bowl and season with salt and pepper.
3. Heat a non-stick pan over medium heat and add a small amount of butter or oil.
4. Pour in the beaten eggs and let them cook for a minute or two.
5. Add the spinach and other fillings to one half of the omelette.
6. When the omelette is almost set, fold the other half over the filling.
7. Cook for another minute or until the omelette is cooked to your desired doneness.
8. Slide the omelette onto a plate and enjoy!

This recipe is quick, easy, and packed with n

In [264]:
test_query_response_4_basic_rag = qdrant_rag_chain_basic.invoke(test_query_4)
print(test_query_response_4_basic_rag)

Based on the provided context, I would recommend the "Healthy Shrimp Stir Fry for Two" or the "My Favorite Fish Recipe" for a healthy dinner option that meets your criteria.

The Healthy Shrimp Stir Fry for Two is a one-dish meal that takes only 16 minutes to prepare and cook. Each serving contains 371.6 calories, which is well under the 500-calorie limit you specified. It's a quick and easy recipe that includes shrimp as the main protein source.

Alternatively, the My Favorite Fish Recipe is another excellent option. It takes 35 minutes to prepare and cook, which falls within your 40-minute time frame. Each serving contains 298 calories, making it a low-calorie choice. This recipe features fish as the main ingredient and is rated as healthy, easy, and suitable for beginner cooks.

Both recipes are healthy, include fish, are under 500 calories per serving, and can be prepared in less than 40 minutes, meeting all your requirements.

Here are the instructions for the Healthy Shrimp Stir 

In [265]:
test_query_response_5_basic_rag = qdrant_rag_chain_basic.invoke(test_query_5)
print(test_query_response_5_basic_rag)

Based on the provided context and your dietary requirements, I would recommend the "Rice Noodle Bake" recipe. Here are the details:

Recipe: Rice Noodle Bake
Prep Time: 10 minutes
Cook Time: 1 hour
Total Time: 1 hour 10 minutes
Servings: 6

This recipe meets your criteria as it is dairy-free, relatively low in sodium (457.9 mg per serving), and can be prepared in under an hour. Additionally, it is categorized as a low-protein dish, which aligns with the ketogenic diet.

Ingredients:
- Rice noodles
- Vegetables (e.g., bell peppers, onions, mushrooms)
- Eggs
- Coconut milk or other dairy-free milk alternative
- Seasonings (e.g., garlic, herbs, spices)

Instructions:
1. Cook the rice noodles according to package instructions.
2. Sauté the vegetables in a pan with a little oil or cooking spray.
3. In a separate bowl, whisk together the eggs and coconut milk (or dairy-free milk alternative). Season with desired herbs and spices.
4. In a baking dish, layer the cooked noodles, sautéed vegetab

In [266]:
test_query_response_6_basic_rag = qdrant_rag_chain_basic.invoke(test_query_6)
print(test_query_response_6_basic_rag)

Based on the provided context and your requirements, I would recommend the "Salmon in a Light, Fragrant Broth" recipe. Here are the details:

Recipe Name: Salmon in a Light, Fragrant Broth
Total Time: 30 minutes
Category: Thai
Keywords: Asian, Kid Friendly, Spicy, Savory, < 30 Mins, Easy, Inexpensive

This recipe meets your criteria as it is a pescatarian main course featuring salmon, uses Asian (Thai) flavors, and can be prepared in under 45 minutes (only 30 minutes total). Additionally, it is relatively low in saturated fat, with only 2.2 grams per serving.

Instructions:
1. Cook Time: 15 minutes
2. Prep Time: 15 minutes
3. Total Time: 30 minutes
4. Servings: 2

While the specific instructions are not provided in the metadata, this Thai-inspired salmon dish cooked in a fragrant broth seems to align well with your requirements for a flavorful, pescatarian main course that can be prepared quickly and is low in saturated fat.

Let me know if you need any other details or have additional

In [267]:
test_query_response_7_basic_rag = qdrant_rag_chain_basic.invoke(test_query_7)
print(test_query_response_7_basic_rag)

Based on the provided context and your requirements, I would recommend the "Apple, Banana, Blueberry Overnight Oats" recipe. Here are the reasons why this recipe fits your criteria:

1. Diabetic-friendly: This recipe is low in sugar (24.7g per serving) and has a good amount of fiber from the oats, fruits, and chia seeds, which can help regulate blood sugar levels.

2. Vegan: The recipe is vegan, containing no animal-derived ingredients.

3. Gluten-free: Oats are naturally gluten-free, making this recipe suitable for those with gluten intolerance or celiac disease.

4. Nut-free: The recipe does not contain any nuts or nut-based ingredients.

5. Low in cholesterol: With only 11.4mg of cholesterol per serving, this recipe is considered low in cholesterol.

6. Rich in omega-3 fatty acids: The chia seeds used in this recipe are an excellent source of omega-3 fatty acids, which are beneficial for heart health and inflammation reduction.

7. Overnight preparation: As the name suggests, this r

In [268]:
test_query_response_8_basic_rag = qdrant_rag_chain_basic.invoke(test_query_8)
print(test_query_response_8_basic_rag)

Based on the provided context and your dietary requirements, I would recommend the "Paleo Chipotle Chicken Cauliflower Bake" recipe. Here's why:

1. Paleo Diet: This recipe is labeled as "Paleo," which means it is grain-free, dairy-free, and follows the principles of the Paleo diet.

2. Dairy-free: The recipe does not contain any dairy products, making it suitable for a dairy-free diet.

3. Gluten-free: Since it is a Paleo recipe, it is naturally gluten-free.

4. Low in Carbs: With only 11.4 grams of carbohydrates per serving, this recipe is considered low in carbs.

5. Relatively Low in Sodium: While not extremely low in sodium, with 159.4 mg per serving, it is lower than some other recipes in the context.

6. Rich in Antioxidants: Cauliflower and chipotle peppers are good sources of antioxidants.

7. Quick Preparation: The total time for this recipe is 50 minutes, which falls within your 30-minute limit.

8. Minimal Cooking Equipment: This recipe can be prepared in a single baking di

### Evaluation method
1. Use LLM as a judge 
2. Basic Sniff test

### Evaluation pipeline
1. Look at EDA result and determine the type of cuisine preesnt in the data. Come up with 8 "Test Questions". - Done. 
    test_query_1 to test_query_8
2. Feed the test questions in to the basic RAG, function calling RAG
3. Gather response and use the following two methods to evaluate them.
   1. LLM as a judge (provide grading ruberic) and ask the eval LLM to provide a score of 1-5.
   2. Sniff test 
  

### Recipe Grading Criteria - Pass into the Eval LLM as part of the prompt

Grading Scale (1-5)

5 - Exceptional Recipe:

    1. Accuracy: The recipe is highly accurate and closely matches the user's query, including all specified ingredients, dietary restrictions, and desired cuisine type.
    2. Clarity: The instructions are clear, easy to follow, and logically sequenced. Cooking times and temperatures are precise.
    3. Creativity: The recipe demonstrates creativity, offering a unique or interesting twist on a classic dish or a novel combination of ingredients.
    4. Completeness: The recipe includes all necessary details, such as ingredient measurements, preparation steps, serving suggestions, and any relevant tips or variations.
    5. Healthiness: The recipe provides a balanced nutritional profile, aligning with any specified health goals or dietary considerations.
    6. User Feedback: The recipe is likely to receive high ratings from users for both taste and ease of preparation.


4 - Very Good Recipe:

    1. Accuracy: The recipe mostly matches the user's query with minor deviations or substitutions that still align with the user's dietary restrictions and preferences.
    2. Clarity: The instructions are clear and easy to follow, with only minor areas that could benefit from additional detail.
    3. Creativity: The recipe shows some creativity and presents an appealing dish, though it may not be as unique as a 5-rated recipe.
    4. Completeness: The recipe includes most necessary details, but might miss a few minor tips or variations.
    5. Healthiness: The recipe is generally healthy, though it may not be as nutritionally balanced as a 5-rated recipe.
    6. User Feedback: The recipe is likely to receive good ratings from users, being tasty and reasonably easy to prepare.

3 - Good Recipe:

    1. Accuracy: The recipe has a reasonable match with the user's query but may include some inaccuracies or ingredient substitutions that slightly alter the dish's nature.
    2. Clarity: The instructions are generally clear but may have a few confusing steps or lack detailed guidance in some areas.
    3. Creativity: The recipe is standard with minimal creativity or uniqueness.
    4. Completeness: The recipe includes the essential details but lacks additional helpful information or suggestions.
    5. Healthiness: The recipe is moderately healthy but may lack balance in terms of nutritional profile.
    6. User Feedback: The recipe is expected to receive average ratings, being satisfactory but not outstanding in taste or ease of preparation.

2 - Fair Recipe:

    1. Accuracy: The recipe has noticeable discrepancies from the user's query, potentially including ingredients that were supposed to be excluded due to dietary restrictions.
    2. Clarity: The instructions are unclear or difficult to follow, with significant gaps or ambiguities.
    3. Creativity: The recipe lacks creativity and may appear bland or uninspired.
    4. Completeness: The recipe is missing several important details, such as precise measurements or key preparation steps.
    5. Healthiness: The recipe is not particularly healthy and may have an unbalanced nutritional profile.
    6. User Feedback: The recipe is likely to receive below-average ratings due to issues with taste, clarity, or preparation difficulty.

1 - Poor Recipe:

    1. Accuracy: The recipe significantly deviates from the user's query, ignoring key dietary restrictions or preferences.
    2. Clarity: The instructions are confusing, incomplete, or incorrect, making the recipe difficult or impossible to follow.
    3. Creativity: The recipe is not creative and may seem haphazard or poorly thought out.
    4. Completeness: The recipe is missing critical details, such as major ingredients, steps, or cooking times.
    5. Healthiness: The recipe is unhealthy and lacks a balanced nutritional profile.
    6. User Feedback: The recipe is likely to receive low ratings due to poor taste, difficulty in preparation, or failure to meet user expectations.

In [116]:
# We will need to tune these prompts
# query_bedrock_llm() definition NEEDS TO BE RERUN
# each time when changes are made to this prompt

eval_message_example = f"""
You are a helpful assistant and expert in reviewing cooking recipes.

Please look at the given user query, recipe generated by another LLM and follow the ruberic below. Provide a score between 1-5 on how good the receipe is.

user query : "{test_query_1}"

Recipe generated by another LLM :"{test_query_response_1}"

Recipe review ruberic: 

Grading Scale (1-5)

5 - Exceptional Recipe:

1. Accuracy: The recipe is highly accurate and closely matches the user's query, including all specified ingredients, dietary restrictions, and desired cuisine type.
2. Clarity: The instructions are clear, easy to follow, and logically sequenced. Cooking times and temperatures are precise.
3. Creativity: The recipe demonstrates creativity, offering a unique or interesting twist on a classic dish or a novel combination of ingredients.
4. Completeness: The recipe includes all necessary details, such as ingredient measurements, preparation steps, serving suggestions, and any relevant tips or variations.
5. Healthiness: The recipe provides a balanced nutritional profile, aligning with any specified health goals or dietary considerations.
6. User Feedback: The recipe is likely to receive high ratings from users for both taste and ease of preparation.
4 - Very Good Recipe:

1. Accuracy: The recipe mostly matches the user's query with minor deviations or substitutions that still align with the user's dietary restrictions and preferences.
2. Clarity: The instructions are clear and easy to follow, with only minor areas that could benefit from additional detail.
3. Creativity: The recipe shows some creativity and presents an appealing dish, though it may not be as unique as a 5-rated recipe.
4. Completeness: The recipe includes most necessary details, but might miss a few minor tips or variations.
5. Healthiness: The recipe is generally healthy, though it may not be as nutritionally balanced as a 5-rated recipe.
6. User Feedback: The recipe is likely to receive good ratings from users, being tasty and reasonably easy to prepare.
3 - Good Recipe:

1. Accuracy: The recipe has a reasonable match with the user's query but may include some inaccuracies or ingredient substitutions that slightly alter the dish's nature.
2. Clarity: The instructions are generally clear but may have a few confusing steps or lack detailed guidance in some areas.
3. Creativity: The recipe is standard with minimal creativity or uniqueness.
4. Completeness: The recipe includes the essential details but lacks additional helpful information or suggestions.
5. Healthiness: The recipe is moderately healthy but may lack balance in terms of nutritional profile.
6. User Feedback: The recipe is expected to receive average ratings, being satisfactory but not outstanding in taste or ease of preparation.
2 - Fair Recipe:

1. Accuracy: The recipe has noticeable discrepancies from the user's query, potentially including ingredients that were supposed to be excluded due to dietary restrictions.
2. Clarity: The instructions are unclear or difficult to follow, with significant gaps or ambiguities.
3. Creativity: The recipe lacks creativity and may appear bland or uninspired.
4. Completeness: The recipe is missing several important details, such as precise measurements or key preparation steps.
5. Healthiness: The recipe is not particularly healthy and may have an unbalanced nutritional profile.
6. User Feedback: The recipe is likely to receive below-average ratings due to issues with taste, clarity, or preparation difficulty.
1 - Poor Recipe:

1. Accuracy: The recipe significantly deviates from the user's query, ignoring key dietary restrictions or preferences.
2. Clarity: The instructions are confusing, incomplete, or incorrect, making the recipe difficult or impossible to follow.
3. Creativity: The recipe is not creative and may seem haphazard or poorly thought out.
4. Completeness: The recipe is missing critical details, such as major ingredients, steps, or cooking times.
5. Healthiness: The recipe is unhealthy and lacks a balanced nutritional profile.
6. User Feedback: The recipe is likely to receive low ratings due to poor taste, difficulty in preparation, or failure to meet user expectations.

"""

In [144]:
# Eval framework
MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"

def query_bedrock_eval_llm(messages):
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'messages': messages,
            'max_tokens': 3000,

            # This config forces the model to always call the recipe db query tool atleast once 
            # https://docs.anthropic.com/en/docs/build-with-claude/tool-use#controlling-claudes-output
            # "tool_choice": {
            #     "type": "tool",
            #     "name": recipe_db_query_tool['name']
            # },
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9,
            'top_k' : 2,
        })
    )
    response_body = json.loads(response.get('body').read())
    
    return response_body

In [145]:
qdrant_rag_chain_eval = (
    RunnableMap(
        # {"context": qdrant_retriever | format_docs,
         {"query": RunnablePassthrough()}
    )
    | process_prompt
    | query_bedrock_eval_llm
    # | parse_event_stream
)

In [148]:
qdrant_rag_chain_eval.invoke(eval_message_example)['content'][0]['text']

"Based on the provided recipe and the review rubric, I would rate the Vegetarian Thai Curry recipe a 4 out of 5.\n\nRationale:\n\n1. Accuracy: The recipe closely matches the user's query, including the request for a vegetarian Asian-inspired dish. All the specified ingredients and dietary restrictions are met.\n\n2. Clarity: The instructions are clear and easy to follow, with logical sequencing of the preparation steps. The cooking times and temperatures are provided.\n\n3. Creativity: The recipe demonstrates some creativity by incorporating Thai flavors and a mix of vegetables, chickpeas, and tofu. While not entirely unique, it offers an appealing and flavorful vegetarian dish.\n\n4. Completeness: The recipe includes all the necessary details, such as ingredient measurements, preparation steps, and serving suggestions. It also provides a few relevant tips.\n\n5. Healthiness: The recipe appears to be generally healthy, with a balanced nutritional profile that aligns with the vegetarian

In [169]:
for i in range(1, 9):
    
    query = locals()[f"test_query_{i}"]
    response = locals()[f"test_query_response_{i}"]
    
    eval_message = f"""
    You are a helpful assistant and expert in reviewing cooking recipes.

    Please look at the given user query, recipe generated by another LLM and follow the rubric below. Provide a score between 1-5 on how good the recipe is.

    user query : "{query}"

    Recipe generated by another LLM : "{response}"

    Recipe review rubric: 

    Grading Scale (1-5)

    5 - Exceptional Recipe:

    1. Accuracy: The recipe is highly accurate and closely matches the user's query, including all specified ingredients, dietary restrictions, and desired cuisine type.
    2. Clarity: The instructions are clear, easy to follow, and logically sequenced. Cooking times and temperatures are precise.
    3. Creativity: The recipe demonstrates creativity, offering a unique or interesting twist on a classic dish or a novel combination of ingredients.
    4. Completeness: The recipe includes all necessary details, such as ingredient measurements, preparation steps, serving suggestions, and any relevant tips or variations.
    5. Healthiness: The recipe provides a balanced nutritional profile, aligning with any specified health goals or dietary considerations.
    6. User Feedback: The recipe is likely to receive high ratings from users for both taste and ease of preparation.
    4 - Very Good Recipe:

    1. Accuracy: The recipe mostly matches the user's query with minor deviations or substitutions that still align with the user's dietary restrictions and preferences.
    2. Clarity: The instructions are clear and easy to follow, with only minor areas that could benefit from additional detail.
    3. Creativity: The recipe shows some creativity and presents an appealing dish, though it may not be as unique as a 5-rated recipe.
    4. Completeness: The recipe includes most necessary details, but might miss a few minor tips or variations.
    5. Healthiness: The recipe is generally healthy, though it may not be as nutritionally balanced as a 5-rated recipe.
    6. User Feedback: The recipe is likely to receive good ratings from users, being tasty and reasonably easy to prepare.
    3 - Good Recipe:

    1. Accuracy: The recipe has a reasonable match with the user's query but may include some inaccuracies or ingredient substitutions that slightly alter the dish's nature.
    2. Clarity: The instructions are generally clear but may have a few confusing steps or lack detailed guidance in some areas.
    3. Creativity: The recipe is standard with minimal creativity or uniqueness.
    4. Completeness: The recipe includes the essential details but lacks additional helpful information or suggestions.
    5. Healthiness: The recipe is moderately healthy but may lack balance in terms of nutritional profile.
    6. User Feedback: The recipe is expected to receive average ratings, being satisfactory but not outstanding in taste or ease of preparation.
    2 - Fair Recipe:

    1. Accuracy: The recipe has noticeable discrepancies from the user's query, potentially including ingredients that were supposed to be excluded due to dietary restrictions.
    2. Clarity: The instructions are unclear or difficult to follow, with significant gaps or ambiguities.
    3. Creativity: The recipe lacks creativity and may appear bland or uninspired.
    4. Completeness: The recipe is missing several important details, such as precise measurements or key preparation steps.
    5. Healthiness: The recipe is not particularly healthy and may have an unbalanced nutritional profile.
    6. User Feedback: The recipe is likely to receive below-average ratings due to issues with taste, clarity, or preparation difficulty.
    1 - Poor Recipe:

    1. Accuracy: The recipe significantly deviates from the user's query, ignoring key dietary restrictions or preferences.
    2. Clarity: The instructions are confusing, incomplete, or incorrect, making the recipe difficult or impossible to follow.
    3. Creativity: The recipe is not creative and may seem haphazard or poorly thought out.
    4. Completeness: The recipe is missing critical details, such as major ingredients, steps, or cooking times.
    5. Healthiness: The recipe is unhealthy and lacks a balanced nutritional profile.
    6. User Feedback: The recipe is likely to receive low ratings due to poor taste, difficulty in preparation, or failure to meet user expectations.
    
    The response format should be the following :
    
    User Query : 
    Recipe reviewed : 
    Score : 
    Reasoning : 
        1. Accuracy:
        2. Clarity :
        3. Creativity :
        4. Completeness : 
        5. Healthiness : 
        6. User Feedback :
    """
    
    response = qdrant_rag_chain_eval.invoke(eval_message)['content'][0]['text']
    print(response)
    print("========")

    

User Query: I enjoy asian fusion food and I am a vegetarian. Give me one recipe with ingredients and instructions.

Recipe reviewed: Vegetarian Thai Curry

Score: 4

Reasoning:

1. Accuracy: The recipe closely matches the user's query, providing a vegetarian Asian-inspired dish as requested. All the specified ingredients and dietary restrictions are accurately addressed.

2. Clarity: The instructions are clear, easy to follow, and logically sequenced. Cooking times and temperatures are provided.

3. Creativity: The recipe demonstrates creativity by offering a unique Thai curry dish, which is an interesting twist on a classic Asian cuisine.

4. Completeness: The recipe includes all necessary details, such as ingredient measurements, preparation steps, serving suggestions, and relevant tips.

5. Healthiness: The recipe provides a balanced nutritional profile, aligning with the user's vegetarian dietary needs.

6. User Feedback: The recipe is likely to receive high ratings from users for 

In [170]:
eval_message

'\n    You are a helpful assistant and expert in reviewing cooking recipes.\n\n    Please look at the given user query, recipe generated by another LLM and follow the rubric below. Provide a score between 1-5 on how good the recipe is.\n\n    user query : "\nI am following a strict paleo diet and need a lunch recipe that is dairy-free, \ngluten-free, low in carbs, and low in sodium. Additionally, it should be rich in antioxidants, \nand can be made in under 30 minutes with minimal cooking equipment.\n"\n\n    Recipe generated by another LLM : "\n[Model]: Based on the recipe metadata retrieved, here is a recommended paleo-friendly, low-carb, low-sodium lunch recipe that is quick and easy to make:\n\nPaleo Sandwich Bread\n\nIngredients:\n- 3 eggs\n- 1/2 cup almond flour\n- 1/4 cup coconut flour \n- 1/4 cup olive oil\n- 1 tsp baking soda\n- 1/4 tsp salt\n\nInstructions:\n1. Preheat oven to 350°F. Grease a 9x5 inch loaf pan.\n2. In a medium bowl, whisk the eggs. Then mix in the almond flou